In [1]:
import pandas as pd
import numpy as np
import math
import QuantLib as ql

## Bootstrapping the yield curve

In [2]:
def print_curve(xlist,ylist, precision  = 4):
    """
    Way to print the curve nicely
    """
    print("__________________ \n")
    print("Maturities Curve ")
    print("__________________")
    
    for x,y in zip(xlist,ylist):
        print(x,"\t\t", round(y,precision))
        
    print("__________________")

In [3]:
#Deposit rates

deposit_maturities = [ql.Period(6,ql.Months), ql.Period(12,ql.Months)]
deposit_rates = [5.25,5.5]

#Bond Rates
bond_maturities = [ql.Period(6*i,ql.Months) for i in range(3,21)]
bond_rates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15,
              7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]

print_curve(deposit_maturities+bond_maturities, deposit_rates+bond_rates)


__________________ 

Maturities Curve 
__________________
6M 		 5.25
1Y 		 5.5
1Y6M 		 5.75
2Y 		 6.0
2Y6M 		 6.25
3Y 		 6.5
3Y6M 		 6.75
4Y 		 6.8
4Y6M 		 7.0
5Y 		 7.1
5Y6M 		 7.15
6Y 		 7.2
6Y6M 		 7.3
7Y 		 7.35
7Y6M 		 7.4
8Y 		 7.5
8Y6M 		 7.6
9Y 		 7.6
9Y6M 		 7.7
10Y 		 7.8
__________________


In [4]:
# constants
# assume for the sake of example that some of the constants
# are the same for depo rates and bond rates
calc_date = ql.Date(15,1,2015)
ql.Settings.instance().evaluationDate = calc_date

calendar = ql.UnitedStates()
business_convention = ql.Unadjusted
day_count = ql.Thirty360()
end_of_month = True
settlement_days = 0
face_value = 100
cupon_frequency = ql.Period(ql.Semiannual)

In [5]:
# Use the deposit rates and bond rates to create individual helpers. 
# Then use the combination of the two helpers to construct the yield curve.

In [6]:
deposit_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                  m,
                  settlement_days,
                  calendar,
                  business_convention,
                  end_of_month,
                  day_count)
                  for r,m in zip(deposit_rates,deposit_maturities)]

In [7]:
bond_helpers = []
for r,m in zip(bond_rates,bond_maturities):
    termination_date = calc_date+m
    schedule = ql.Schedule(calc_date,
                          termination_date,
                          cupon_frequency,
                          calendar,
                          business_convention,
                          business_convention,
                          ql.DateGeneration.Backward,
                          end_of_month)
    helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(face_value)),
                                   settlement_days,
                                   face_value,
                                   schedule,
                                   [r/100.0],
                                   day_count,
                                   business_convention)
    bond_helpers.append(helper)

In [8]:
rate_helpers = deposit_helpers + bond_helpers
yield_curve = ql.PiecewiseLogCubicDiscount(calc_date,rate_helpers,day_count)

In [9]:
spots = []
tenors = []

for d in yield_curve.dates():
    yrs = day_count.yearFraction(calc_date,d)
    compounding = ql.Compounded
    freq = ql.Semiannual
    zero_rate = yield_curve.zeroRate(yrs,compounding,freq)
    tenors.append(yrs)
    eq_rate = zero_rate.equivalentRate(day_count,
                                      compounding,
                                      freq,
                                      calc_date,
                                      d).rate()
    spots.append(100*eq_rate)

In [10]:
print("TENORS__________________SPOTS")
for i in range(0,len(spots)):
    print(tenors[i],"____________________",round(spots[i],3))

TENORS__________________SPOTS
0.0 ____________________ 0.0
0.5 ____________________ 5.25
1.0 ____________________ 5.426
1.5 ____________________ 5.761
2.0 ____________________ 6.02
2.5 ____________________ 6.283
3.0 ____________________ 6.55
3.5 ____________________ 6.822
4.0 ____________________ 6.87
4.5 ____________________ 7.095
5.0 ____________________ 7.205
5.5 ____________________ 7.257
6.0 ____________________ 7.31
6.5 ____________________ 7.429
7.0 ____________________ 7.485
7.5 ____________________ 7.543
8.0 ____________________ 7.671
8.5 ____________________ 7.802
9.0 ____________________ 7.791
9.5 ____________________ 7.929
10.0 ____________________ 8.072
